EDA посмотрел тут:

https://medium.com/@ravitee/santander-product-recommendation-ee4122d15072

https://medium.com/@samarthjoelram/santander-recommendation-system-cab6b40596b5

На ресерч, подбор параметров и тд времени мало, потому выбираю по интуиции. Только обучение берта на 3 эпохах занимат 15 часов

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMaskedLM, BertConfig   # Выбрал берт, тк раньше работал с ним, хоть и для других задач
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt

Загружаем данные

In [ ]:
data_path = "/home/dhauryk/!DUNAI/train_ver2.csv"
df = pd.read_csv(data_path)
#df = df[-100000:]

Нахожу series с продуктами

In [ ]:
product_columns = [col for col in df.columns if col.startswith("ind_")]

Функция для создания последовательности с продуктами

In [ ]:
def convert_to_sequence(row):
    res_convert = " ".join([col for col in product_columns if row[col] == 1])
    
    return res_convert

В тестовом наборе данных нету реальных меток. Для рассчета MAP7 отделяем тестовый набор из данных для обучения. Поскольку тестовый набор данных содержит будущий период, то сортируем свой тест и по дате

In [ ]:
def preprocess_data(df):
    df = df.sort_values(["ncodpers", "fecha_dato"])  # Сортируем по идентификатору клиента (ncodpers) и дате
    df["product_sequence"] = df.apply(convert_to_sequence, axis=1)  # Создаем последовательности продуктов для каждого клиента
    unique_sequences = df.groupby("ncodpers")["product_sequence"].apply(lambda x: " ".join(x)).reset_index()  # Удаляем дублирующиеся последовательности для каждого клиента
    
    return unique_sequences

data = preprocess_data(df)

Пока не перемешиваем, чтобы не потерять сортировку по датам

In [ ]:
train_data, temp_data = train_test_split(data, test_size=0.3, shuffle=False, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.3, shuffle=False, random_state=42)

Загружаем токинайзер и токенизируем данные для train, val, test

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_data['product_sequence']), padding=True, truncation=True, return_tensors="tf")
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_encodings['input_ids']))
train_dataset = train_dataset.shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)

val_encodings = tokenizer(list(val_data['product_sequence']), padding=True, truncation=True, return_tensors="tf")
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_encodings['input_ids']))
val_dataset = val_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

test_encodings = tokenizer(list(test_data['product_sequence']), padding=True, truncation=True, return_tensors="tf")
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_encodings['input_ids']))
test_dataset = test_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

Подгружаем модель и задаем конфиги. Конфиги меньше дефолтных ради экономии ресурсов и времени на обучении

In [ ]:
config = BertConfig.from_pretrained("bert-base-uncased", num_hidden_layers=4, num_attention_heads=4, hidden_size=256)
model = TFBertForMaskedLM(config)

Задаем оптимизатор, лосс. Компилируем модель

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # 2 и более меток классов, потому SparseCategoricalCrossentropy
model.compile(optimizer=optimizer, loss=loss)

Запускаем обучение и пишем историю для просмотра, чтобы исключить подгон под валидацию

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)   # Берту обычно хватает 2-4 эпохи

Смотрим лосс на обучении и валидации

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss Over Epochs')
plt.show()

Подаем в модель тестовые данные, сортируем и берем топ 7

In [ ]:
k = 7
predictions = model.predict(test_encodings['input_ids']).logits
predicted_tokens = np.argsort(-predictions, axis=-1)[:, :, :k]

Функция для подсчета MAP7

In [ ]:
def map7(preds, actuals, k=7):
    avg_precisions = []  # Сюда собираем средние точности каждой последовательности
    for pred, actual in zip(preds, actuals):
        match = 0  # Счетчик совпадений
        sum_precisions = 0  # Сумма точности в каждом ранге
        for i, p in enumerate(pred[:k]):  # Берем топ 7
            if p in actual:  # Проверяем есть ли прогноз в актуальном списке
                match += 1
                sum_precisions += match / (i + 1)  # Точность вычисления на i-м ранге
        avg_precisions.append(sum_precisions / min(k, len(actual)))  # Считаем среднюю точность

    return np.mean(avg_precisions)  # Возвращаем среднее значения всех средних значений точности



Преобразовываем реальные последовательности продуктов в идентификаторы токенов

In [ ]:
actual_tokens = [tokenizer.encode(seq, add_special_tokens=False) for seq in test_data['product_sequence']]
map7_score = map7(predicted_tokens, actual_tokens, k)
print(f"MAP@7 Score: {map7_score:.4f}")

Сохраняем модель и токинайзер

In [ ]:
model.save_pretrained("bert4rec")
tokenizer.save_pretrained("bert4rec")